In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import glob
import json
import pickle
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import Counter
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from io import BytesIO
import base64
from country_list import countries_for_language
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package punkt to /Users/dhv001/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
import warnings

warnings.filterwarnings(module='sklearn*', action='ignore', category=DeprecationWarning)

In [13]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 100)
pd.set_option('display.max_colwidth', None)

In [14]:
DATA_FILE = 'labeled_npi.csv'

In [15]:
# data.pkl from Kaggle notebook
df = pd.read_pickle('data.pkl')

In [16]:
len(df)

1612

In [17]:
def pca_apply(df, columns, n_comp):
    new_df = df.copy()
    for col in columns:
        pca = PCA(n_components=n_comp, random_state=1)
        new_df[col+'_pca'] = list(pca.fit_transform(np.stack(df[col].to_numpy())))
    return new_df.reset_index(drop=True)

def apply_scaler(df, columns):
    new_df = df.copy()
    for col in columns:
        scaler = StandardScaler()
        new_df[col + '_scaled'] = list(scaler.fit_transform(np.stack(df[col].to_numpy())))
    return new_df.reset_index(drop=True)

In [24]:
class Preprocesser:
    def __init__(self, df):
        self.df_labels = pd.read_csv(DATA_FILE)
        self.keywords = ['incident command system',
                         'emergency operations',
                         'joint information center',
                         'social distancing',
                         'childcare closers',
                         'travel advisory',
                         'travel warning',
                         'isolation',
                         'quarantine',
                         'mass gathering cancellations',
                         'school closures',
                         'facility closures',
                         'evacuation',
                         'relocation',
                         'restricting travel',
                         'travel ban',
                         'patient cohort',
                         'npi']
        self.occurances_minimum = 2
        self.df_full = df
        self.countries = set([k.lower() for k in dict(countries_for_language('en')).values()])
        print(self.df_full.shape)
        self.key_slice()
        print(self.df_full.shape)

    def run_process(self):
        self.remove_punc(['body_text','abstract'])
        self.remove_stopwords(['body_text', 'abstract'])
        self.to_tfidf(['body_text', 'abstract'])
        print('Running XG Boost Model')
        self.npi_model()
        print('Model Complete')
        self.df_full = self.df_full.merge(self.df_labels, on="title", how="inner")
        self.df_full = self.df_full.loc[self.df_full.isNPI.notna()]
        self.df_full = pca_apply(self.df_full, ['abstract_tfidf','body_text_tfidf'], 10)
        self.df_full = apply_scaler(self.df_full,['abstract_tfidf_pca','body_text_tfidf_pca'])
        print('NPI Slicing')
        self.npi_slice()
        self.set_country_columns()
        print(self.df_full.shape)
        
    def set_country_columns(self):
        def get_country(row):
            text_set = set(row['body_text'].split(' '))
            return list(self.countries.intersection(text_set))
        self.df_full['countries'] = self.df_full.apply(get_country, axis=1)
    
    def key_slice(self):
        self.df_full = self.df_full[self.df_full['abstract'].str.contains('|'.join(self.keywords), na=False, regex=True)].reset_index(drop=True)
        
    def npi_slice(self):
        def get_count(row):
            return sum([row['abstract'].count(keyword) for keyword in self.keywords])
        self.df_full = self.df_full[self.df_full.apply(get_count, axis=1) >= self.occurances_minimum]
        
    def remove_stopwords(self,columns):
        stop = stopwords.words('english')
        for col in columns:
            self.df_full[col] = self.df_full[col].astype(str).apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

    def to_tfidf(self, columns):
        for col in columns:
            tfidfv = TfidfVectorizer()
            self.df_full[col + '_tfidf'] = list(tfidfv.fit_transform(self.df_full[col]).toarray())
            
    def remove_punc(self, columns):
        for col in columns:
            self.df_full[col] = self.df_full[col].str.replace('[^a-zA-Z\s]+','')
            
    def npi_model(self):
        df = self.df_full.copy()
        df = df.merge(self.df_labels, on="title", how="inner")
        df = df.loc[df.isNPI.notna()]
        pca_df = pca_apply(df, ['abstract_tfidf','body_text_tfidf'], 10)
        scaled_df = apply_scaler(pca_df,['abstract_tfidf_pca','body_text_tfidf_pca'])
        X = np.stack(scaled_df['body_text_tfidf_pca_scaled'].to_numpy())
        y = scaled_df["isNPI"]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10, stratify=y)
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dtest = xgb.DMatrix(X_test, label=y_test)
        param = {'max_depth': 2, 'eta': 1, 'objective': 'binary:logistic', 'eval_metric': 'auc'}
        self.clf_xgb = xgb.XGBClassifier(max_depth=6, learning_rate=0.1,silent=False, objective='binary:logistic', \
                          booster='gbtree', n_jobs=8, nthread=None, gamma=0, min_child_weight=1, max_delta_step=0, \
                          subsample=0.8, colsample_bytree=0.8, colsample_bylevel=1, reg_alpha=0, reg_lambda=1)
        self.clf_xgb.fit(X_train, y_train)
        y_pred = self.clf_xgb.predict(X_test)
        precision_recall_fscore_support(y_test, y_pred, average='macro')
        score = accuracy_score(y_test, y_pred)
        print(f'Accuracy Score: {score}')
        
    def npi_slice(self):
        def npi_col(row):
            x = [row['body_text_tfidf_pca_scaled']]
            y_pred = self.clf_xgb.predict(x)[0]
            if y_pred > 0:
                return True
            return False
        self.df_full['npi_pred'] = self.df_full.apply(npi_col, axis=1)
        self.df_full = self.df_full[self.df_full['npi_pred']].reset_index(drop=True)
        remove_list = ['body_text_tfidf',
               'abstract_tfidf',
               'Unnamed: 0',
               'Unnamed: 0.1',
               'abstract_tfidf_pca',
               'body_text_tfidf_pca',
               'npi_pred']
        self.df_full = self.df_full.drop(columns=remove_list)
        
    

In [25]:
prepr = Preprocesser(df)

(1612, 6)
(1612, 6)


In [26]:
prepr.run_process()

Running XG Boost Model
Accuracy Score: 0.8442622950819673
Model Complete
NPI Slicing
(183, 10)


In [27]:
cluster_df = prepr.df_full


In [40]:
def reduce_dimension(row):
    return row[:3]

In [42]:
cluster_df['abstract_tfidf_pca_scaled'] = cluster_df['abstract_tfidf_pca_scaled'].apply(reduce_dimension)
cluster_df['body_text_tfidf_pca_scaled'] = cluster_df['body_text_tfidf_pca_scaled'].apply(reduce_dimension)

In [43]:
cluster_df.head()

paper_id  \
0  14d04f36cb13550aa7769b61a079fa54031a21eb   
1  59e70474f7eae42943ba63363f887cdaa91e29f9   
2  152b9f61013ab57a8a696b2cdbdd696dea14a941   
3  8bf581553621ca6dd31516239bb2edb5389d6a30   
4  d6d6e6d63b14f7793d2e9d612d8df489438766bc   

                                                                                                                                                                                             title  \
0                                                                                                                                                           risk management and health care policy   
1                                                                                         pandemic h1n1 in canada and the use of evidence in developing public health policies e a policy analysis   
2                                                                        experience of severe acute respiratory syndrome in singapore: importation of cases, and defense strategies at the airport   
3                                                                                                                         variation in health care worker removal of personal protective equipment   
4  selected topics: disaster medicine implementation of the hospital emergency incident command system during an outbreak of severe acute respiratory syndrome (sars) at a hospital in taiwan, roc   

                                                                                                                                                              author_list  \
0                                                                               [Melanie Taylor, Beverley Raphael, Margo Barr, Kingsley Agho, Garry Stevens, Louisa Jorm]   
1  [Laura C Rosella, Kumanan Wilson, Natasha S Crowcroft, Anna Chu, Ross Upshur, Donald Willison, Shelley L Deeks, Brian Schwartz, Jordan Tustin, Doug Sider, Vivek Goel]   
2                                                                                                                  [Annelies Wilder-Smith, Kee Tai Goh, Nicholas I Paton]   
3                                                                                                                    [Caroline Zellmer, Sarah Van Hoof Bsn, Nasia Safdar]   
4                                                                      [Ming-Che Tsai, Jeffrey L Arnold, Chia-Chang Chuang, Chih-Hsien Chi, Ching-Chuan Liu, Yu-Jen Yang]   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [44]:
def tokenize(row):
    title_tokens = []
    title = row['abstract']
    if title == title:
        title = re.sub('(/|\|:|&|#|-|\.)', '', title)
        tokens = word_tokenize(title)
        remove_sw = [word for word in tokens if word not in stopwords.words('english')]
        remove_numbers = [word for word in remove_sw if not word.isnumeric()]
        remove_comas = [word for word in remove_numbers if not word in [',', '(', ')', '"', ':', '``', '.', '?']]
        title_tokens.extend(remove_comas)
    return [value[0] for value in Counter(title_tokens).most_common()[0:30]]

In [45]:
cluster_df['tokens'] = cluster_df.apply(tokenize, axis=1)

In [47]:
len(cluster_df)

183

In [46]:
# from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate
import plotly.express as px
import plotly.graph_objects as go
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State

def get_breaks(row, col, word_limit=45, break_char='<br>', colon=True):
    col_list = ['tokens', 'author_list', 'doi', 'countries']
    if row[col] == row[col]:
        data = ''
        if col in col_list:
            if col != 'tokens':
                data = f'**{col.capitalize()}:** '
            words = row[col]
        elif colon:
            if break_char == '<br>':
                data = f'{col.capitalize()}: '
            else:
                data = f'**{col.capitalize()}:** '
            words = row[col].split(' ')
        else:
            words = row[col].replace('<br>', '').split(' ')
            words[0] = f'**{words[0]}**'
        total_chars = 0

        # add break every length characters
        for i in range(len(words)):
            total_chars += len(words[i])
            if total_chars > word_limit:
                data += f'{break_char}{words[i]}'
                total_chars = 0
            else:
                if col in col_list and data:
                    data += f' {words[i]},'
                else:
                    data += f' {words[i]}'
        return data.strip().rstrip(',')
    return row[col]

def get_country_df(df):
    country = []
    count = []
    for k in dict(countries_for_language('en')).values():
        len_country = len(df[df['countries'].map(set([k.lower()]).issubset)])
        country.append(k.lower())
        count.append(len_country)
    return pd.DataFrame({'country': country, 'count': count}) 

def display_wordcloud(text):
    wc= WordCloud(max_font_size=50,
                          max_words=100,
                          background_color='white',
                          width=480, height=360).generate(text)
    return wc.to_image()

class Cluster_Plot:
    def __init__(self, df, text_type, clust_nums):
        self.styles = {
            'pre': {
                'border': 'thin lightgrey solid',
                'overflowX': 'scroll'
            }
        }
        self.set_app()
        self.df = df
        self.set_text(text_type)
        self.clust_nums = clust_nums
        self.create_cluster_df()
        
        if self.app is not None and hasattr(self, 'callbacks'):
            self.callbacks(self.app)

    def run_process(self):
        self.set_app_layout()
        self.app.run_server()
        
    def set_app(self):
        self.app = dash.Dash(__name__,
                        external_stylesheets=["https://codepen.io/chriddyp/pen/bWLwgP.css", "../localstyles.css"])
        
    def set_app_layout(self):
        self.app.layout = html.Div(children=[
            html.Div(className='row', style={'background-color': '#142a57'}, children=[
                html.Div(className='container', style={'max-width': 'unset'}, children=[
                    html.Div(className='row', children=[
                        html.Div(className='nine columns', children=[
                            html.H1('NPI Cluster Analysis', style={ 'color': 'white', 'padding-top': '1%'})
                        ]),
                        html.Div(className='three columns', children=[
                        ])
                    ])
                ])
            ]),
            html.Div(className='container', style={'max-width': 'unset'}, children=[
                html.Div(className='row', children=[
                    html.Div(className='three columns', children=[
                        html.H2('Cluster Control Panel'),
                        html.Label('Select to show 2d or 3d visualization'),
                        dcc.RadioItems(
                            id='dimension',
                            options=[{'label': '2d', 'value': '2d'},
                                     {'label': '3d', 'value': '3d'}],
                            value='3d'
                        ),
                        html.Label('Select to cluster on the article abstract or body'),
                        dcc.RadioItems(
                            id='abstract_or_body',
                            options=[{'label': 'abstract', 'value': 'abstract'},
                                     {'label': 'body', 'value': 'body_text'}],
                            value=self.text_type
                        ),
                        html.Label('Select number of Clusters'),
                        dcc.Dropdown(
                            id='cluster_num',
                            options=[{'label': i+1, 'value': i+1} for i in list(range(20))],
                            value=self.clust_nums
                        ),
                        html.Label('Select cluster to view'),
                        dcc.Dropdown(
                            id='cluster_id',
                            options=self.cluster_id_list,
                            value='all'
                        ),
                    ]),
                    html.Div(className='six columns', children=[
                        dcc.Graph(id="graph", style={"width": "90%", "display": "inline-block"})
                    ]),
                    html.Div(className='six columns', children=[
                        html.Button(id='geo-button', children='Reset Map'),
                        dcc.Graph(id="geo-graph", style={"width": "90%", "display": "inline-block"})
                    ]),
                    html.Div(className='three columns', children=[
                        html.H2('Filter Articles'),
                        html.Label('Search keywords'),
                        dcc.Input(id='search',
                            value='',
                            type='text'
                        ),
                        html.Button(id='submit-button-search', children='Submit'),
                        html.H3('Word Cloud for Article'),
                        html.Img(id="image_wc")
                    ]),
                ]),
                html.Div(className='row', children=[
                    html.Div([
                        dcc.Markdown("""
                            **Selected Article**

                            Click on values in the plot to select article.
                        """),
                        dcc.Markdown(id='hover-data', style=self.styles['pre'])
                    ])
                ])
            ]),
            html.Div(className='row', style={'background-color': '#142a57', 'position': 'fixed', 'bottom': '0'}, children=[
                html.Div(style={'max-width': 'unset'}, children=[
                ])
            ])
        ])
        
    def create_cluster_df(self):
        self.cluster_id_list = [{'label': i, 'value': i} for i in list(range(self.clust_nums))]
        self.cluster_id_list.append({'label': 'all', 'value': 'all'})
        new_df = self.df.copy()
        kmeans = KMeans(n_clusters = self.clust_nums, random_state=1)
        new_df[self.col_cluster_id] = list(kmeans.fit_predict(np.stack(new_df[self.col].to_numpy())))
        self.cluster_df = new_df.reset_index(drop=True)
        self.cluster_df['title'] = self.cluster_df.apply(get_breaks, args=('title',), axis=1)
        self.cluster_df[['x', 'y', 'z']] = pd.DataFrame(self.cluster_df[self.col].values.tolist(),
                                                        index = self.cluster_df.index)
    
    def set_text(self, text_type):
        self.text_type = text_type
        self.col = f'{self.text_type}_tfidf_pca_scaled'
        self.col_cluster_id = f'{self.text_type}_tfidf_pca_scaled_clusterID'
        
    def callbacks(self, app):
        @app.callback([Output('cluster_id', 'options'),
                       Output('graph', 'figure'),
                       Output('geo-graph', 'figure')],
                      [Input('dimension', 'value'),
                       Input('abstract_or_body', 'value'),
                       Input('cluster_num', 'value'),
                       Input('cluster_id', 'value'),
                       Input('geo-button', 'n_clicks'),
                       Input("geo-graph", "clickData"),
                       Input('submit-button-search', 'n_clicks')],
                      [State('search', 'value')])
        def update_clusters(dimension, abstract_or_body, cluster_num, cluster_id, geoclicks, geoClickData, clicks, search_string):
            changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]
            self.set_text(abstract_or_body)
            self.clust_nums = cluster_num
            self.create_cluster_df()
            options = self.cluster_id_list
            df = self.cluster_df.copy()
            show_scale = True
            if cluster_id != 'all':
                show_scale = False
                df = df[df[self.col_cluster_id] == cluster_id]
            if 'geo-button' not in changed_id:
                if geoClickData:
                    country = geoClickData['points'][0]['location']
                    df = df[df['countries'].map(set([country]).issubset)]
            if search_string:
                if len(df[df['tokens'].map(set([search_string]).issubset)]):
                    df = df[df['tokens'].map(set([search_string]).issubset)]
            country_df = get_country_df(df)
            if dimension == '2d':
                fig = px.scatter(df, x='x', y='y',
                                 color=self.col_cluster_id,
                                 hover_name='title',
                                 hover_data=['paper_id', 'doi'])
                fig.update_layout(title = '2D cluster of research papers',
                                  xaxis = dict(dtick=1, range=[-5,5], scaleratio = 1),
                                  yaxis = dict(dtick=1, range=[-5,5], scaleratio = 1),
                                  hoverlabel=dict(
                                    bgcolor='white', 
                                    font_size=8, 
                                    font_family='Rockwell'
                                  ),
                                  coloraxis=dict(
                                    colorbar=dict(title='Cluster ID'),
                                    showscale=show_scale
                                  ))
            elif dimension == '3d':
                fig = px.scatter_3d(df, x='x', y='y', z='z',
                                    color=self.col_cluster_id,
                                    hover_name='title',
                                    hover_data=['paper_id', 'doi'])
                fig.update_layout(title = '3D cluster of research papers',
                                  scene = dict(
                                    xaxis = dict(dtick=1, range=[-5,5],),
                                    yaxis = dict(dtick=1, range=[-5,5],),
                                    zaxis = dict(dtick=1, range=[-5,5],),),
                                  hoverlabel=dict(
                                    bgcolor='white', 
                                    font_size=8, 
                                    font_family='Rockwell'
                                  ),
                                  coloraxis=dict(
                                    colorbar=dict(title='Cluster ID'),
                                    showscale=show_scale
                                  ))
            fig2 = px.scatter_geo(country_df,
                                  locationmode='country names',
                                  locations='country',
                                  hover_name='country',
                                  size='count',
                                  projection='natural earth')
            return options, fig, fig2
        
        @app.callback([Output("hover-data", "children"),
                      Output('image_wc', 'src')],
                      [Input("graph", "clickData"),
                       Input('image_wc', 'id')])
        def display_click_data(clickData, b):
            string = None
            if clickData:
                click_paper_id = clickData['points'][0]['customdata'][0]
                click_index = self.cluster_df[self.cluster_df['paper_id'] == click_paper_id].index[0]
                token_string = ''
                country_string = ''
                if self.cluster_df.iloc[click_index]["tokens"]:
                    token_string = get_breaks(self.cluster_df.iloc[click_index],
                                              'tokens',
                                              word_limit=100,
                                              break_char='\n')
                    token_string = f'**Keywords**: {token_string}'
                if self.cluster_df.iloc[click_index]["countries"]:
                    country_string = get_breaks(self.cluster_df.iloc[click_index],
                                                'countries',
                                                word_limit=100,
                                                break_char='\n')
                    
                string = get_breaks(self.cluster_df.iloc[click_index],
                                    'title',
                                    word_limit=100,
                                    break_char='\n',
                                    colon=False)
                item_list = ['abstract', 'body_text', 'author_list', 'paper_id', 'doi']
                for i in item_list:
                    formatted_data = get_breaks(self.cluster_df.iloc[click_index],
                                                i,
                                                word_limit=100,
                                                break_char='\n')
                    string += f'\n\n{formatted_data}'
                img = BytesIO()
                display_wordcloud(self.cluster_df.iloc[click_index][self.text_type]).save(img, format='PNG')
                image = 'data:image/png;base64,{}'.format(base64.b64encode(img.getvalue()).decode())
                if token_string:
                    string = f'{token_string}\n\n{string}'
                if country_string:
                    string = f'{country_string}\n\n{string}'
                return string, image
            return string, None

c = Cluster_Plot(cluster_df, 'abstract', 10)
c.run_process()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Apr/2020 19:17:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2020 19:17:04] "GET /localstyles.css HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2020 19:17:04] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2020 19:17:04] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2020 19:17:04] "GET /_dash-component-suites/dash_core_components/async-plotlyjs.v1_9_1m1586525729.js HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2020 19:17:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2020 19:17:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2020 19:17:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2020 19:17:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2020 19:17:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2020 19:17:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/